In [6]:
import cv2
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sort.tracker import SortTracker


In [7]:
net = cv2.dnn.readNet('../model/vehicle_novo.onnx')
LARGURA_ENTRADA =  640
ALTURA_ENTRADA = 640

In [10]:
def get_detections(img,net):
    # 1.CONVERT IMAGE TO YOLO FORMAT
    image = img.copy()
    row, col, d = image.shape

    max_rc = max(row,col)
    input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
    input_image[0:row,0:col] = image

    # 2. GET PREDICTION FROM YOLO MODEL
    blob = cv2.dnn.blobFromImage(input_image,1/255,(LARGURA_ENTRADA,ALTURA_ENTRADA),swapRB=True,crop=False)
    net.setInput(blob)
    preds = net.forward()
    detections = preds[0]

    return input_image, detections

def non_maximum_supression(input_image,detections):

    # 3. FILTER DETECTIONS BASED ON CONFIDENCE AND PROBABILIY SCORE

    # center x, center y, w , h, conf, proba
    boxes = []
    confidences = []

    image_w, image_h = input_image.shape[:2]
    x_factor = image_w/LARGURA_ENTRADA
    y_factor = image_h/ALTURA_ENTRADA

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4] # confidence of detecting license plate

        if confidence > 0.4:
            class_score = row[5] # probability score of license plate
            if class_score > 0.2:
                cx, cy , w, h = row[0:4]

                left = int((cx - 0.5*w)*x_factor)
                top = int((cy-0.5*h)*y_factor)
                width = int(w*x_factor)
                height = int(h*y_factor)
                box = np.array([left,top,width,height])

                confidences.append(confidence)
                boxes.append(box)

    # 4.1 CLEAN
    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()

    # 4.2 NMS
    index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45)

    return boxes_np, confidences_np, index

def drawings(image,boxes_np,confidences_np,index):
    # 5. Drawings
    for ind in index:
        x,y,w,h =  boxes_np[ind]
        bb_conf = confidences_np[ind]
        conf_text = 'plate: {:.0f}%'.format(bb_conf*100)
        #license_text = extract_text(image,boxes_np[ind])


        cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,255),2)
        cv2.rectangle(image,(x,y-30),(x+w,y),(255,0,255),-1)
        cv2.rectangle(image,(x,y+h),(x+w,y+h+25),(0,0,0),-1)


        cv2.putText(image,conf_text,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),1)
        #cv2.putText(image,license_text,(x,y+h+27),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,0),1)

    return image

def yolo_predictions(img, net, track_id_counter):
    # 1: Detecção
    input_image, detections = get_detections(img, net)
    # 2: Supressão Não Máxima
    boxes_np, confidences_np, index = non_maximum_supression(input_image, detections)

    # Create an array for SORT input
    detections_for_sort = []

    for ind in index:
        x_min, y_min, width, height = boxes_np[ind]  # Extract coordinates and dimensions
        x_max = x_min + width
        y_max = y_min + height

        detection = [x_min, y_min, x_max, y_max, confidences_np[ind], track_id_counter]  # Append track ID
        detections_for_sort.append(detection)

        # Increment the track ID counter for the next frame
        track_id_counter += 1
        
    return input_image, detections_for_sort, track_id_counter

In [11]:
video_path = '../videos/w2.mp4'
tracker = SortTracker()
cap = cv2.VideoCapture(video_path)
track_id_counter = 0
# Loop para processar cada quadro
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Chamar a função de predição do YOLO para processar o quadro
    result_img, detections_for_sort, track_id_counter = yolo_predictions(frame, net, track_id_counter)
    if detections_for_sort != []:
        dets = np.array(detections_for_sort)
        #x_min, y_min, x_max, y_max, conf, track_id = detections_for_sort[0]
    
        online_targets = tracker.update(dets, None)
        print(online_targets)
        for d in online_targets:
            xmin, ymin, xmax, ymax, track_id, _, _ = map(int, d)
            cv2.rectangle(result_img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(result_img, f"ID: {track_id}", (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.namedWindow("TESTE", cv2.WINDOW_NORMAL)
        cv2.imshow("TESTE", result_img)
        cv2.waitKey(0)

    # Mostrar o quadro de saída em uma janela
    


[[4.04000000e+02 0.00000000e+00 6.30000000e+02 4.40000000e+01
  1.00000000e+00 0.00000000e+00 4.98853445e-01]]
[[ 3.98570775e+02 -4.92555175e-01  6.34429325e+02  4.74922555e+01
   1.00000000e+00  0.00000000e+00  4.98853445e-01]]
[[3.92094359e+02 1.22089451e+00 6.34258801e+02 5.34846395e+01
  1.00000000e+00 0.00000000e+00 4.98853445e-01]]
[[3.85813930e+02 1.62714194e+00 6.34303089e+02 5.77594870e+01
  1.00000000e+00 0.00000000e+00 4.98853445e-01]]
[[3.16850842e+02 7.31209629e+00 6.15194483e+02 8.27027004e+01
  1.00000000e+00 0.00000000e+00 4.98853445e-01]]
[[2.96231542e+02 9.54531209e+00 6.09522900e+02 9.75996953e+01
  1.00000000e+00 0.00000000e+00 4.98853445e-01]]
[[2.87438241e+02 8.64036244e+00 6.07631902e+02 1.07079493e+02
  1.00000000e+00 0.00000000e+00 4.98853445e-01]]
[[2.86005224e+02 8.34675456e+00 6.06109877e+02 1.15249833e+02
  1.00000000e+00 0.00000000e+00 4.98853445e-01]]
[[2.87026648e+02 7.41854573e+00 6.05915864e+02 1.22512196e+02
  1.00000000e+00 0.00000000e+00 4.98853445e